In [1]:
import pandas as pd
fish = pd.read_csv('data/Fish.csv')
fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [2]:
fish['Species'].unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [9]:
fish_input = fish[['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']]
fish_target = fish[['Species']]

In [10]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)
#train_scaled[:5]

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
kn.score(train_scaled, train_target)
kn.score(test_scaled, test_target)

/Users/june/Desktop/DMF/ML/venv/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.825

In [20]:
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [21]:
kn.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

In [22]:
kn.predict_proba(test_scaled[:5])

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.33333333,
        0.        , 0.        ]])

In [25]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')

In [27]:
train_bream_smelt = train_scaled[bream_smelt_indexes['Species']]

In [28]:
target_bream_smelt = train_target[bream_smelt_indexes['Species']]

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

/Users/june/Desktop/DMF/ML/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [31]:
lr.predict(train_bream_smelt[:5])

array(['Bream', 'Smelt', 'Bream', 'Bream', 'Bream'], dtype=object)

In [32]:
lr.predict_proba(train_bream_smelt[:5])

array([[0.99793611, 0.00206389],
       [0.02391315, 0.97608685],
       [0.99575505, 0.00424495],
       [0.98765399, 0.01234601],
       [0.99806396, 0.00193604]])

In [33]:
lr.classes_

array(['Bream', 'Smelt'], dtype=object)

In [34]:
lr.coef_

array([[-0.36781969, -0.50365695, -0.52507176, -0.60515406, -0.92933501,
        -0.66853977]])

In [35]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
lr.score(train_scaled, train_target)

/Users/june/Desktop/DMF/ML/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9411764705882353

In [36]:
lr.score(test_scaled, test_target)

0.925

In [50]:
np.round(lr.predict_proba(test_scaled[:5]), decimals=3)

array([[0.   , 0.014, 0.839, 0.   , 0.137, 0.007, 0.003],
       [0.   , 0.002, 0.042, 0.   , 0.006, 0.949, 0.   ],
       [0.   , 0.   , 0.03 , 0.936, 0.01 , 0.024, 0.   ],
       [0.012, 0.034, 0.27 , 0.004, 0.597, 0.   , 0.083],
       [0.   , 0.   , 0.906, 0.002, 0.088, 0.003, 0.001]])

In [39]:
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [40]:
lr.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [43]:
decision = lr.decision_function(test_scaled[:5])

In [47]:
from scipy.special import softmax
import numpy as np

proba = softmax(decision, axis=1)
np.round(proba, decimals=3)

array([[0.   , 0.014, 0.839, 0.   , 0.137, 0.007, 0.003],
       [0.   , 0.002, 0.042, 0.   , 0.006, 0.949, 0.   ],
       [0.   , 0.   , 0.03 , 0.936, 0.01 , 0.024, 0.   ],
       [0.012, 0.034, 0.27 , 0.004, 0.597, 0.   , 0.083],
       [0.   , 0.   , 0.906, 0.002, 0.088, 0.003, 0.001]])